#   **Multimedia systems course | ECE AUTH | Winter semester 2022**
## **Simplified mp3 CODEC**

>   Giachoudis Christos


>   Kostopoulos Andreas Marios


![ logo](mp3.png)

## Necessary imports

In [5]:
# Importing the necessary libraries, files and modules
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.io import wavfile
from frame import *

## h coefficients import


In [14]:
h_coeffs = np.load('h.npy', allow_pickle=True).tolist()
print(type(h_coeffs))
h_coeffs = h_coeffs['h'].reshape(-1,)
# print(h_coeffs[4])
# print(h_coeffs)
print(len(h_coeffs))

<class 'dict'>
512


## Import `wav` file

In [7]:
samplerate, data = wavfile.read('./myfile.wav')
print(f"WAV file imported successfully!!! This file has been created with a sample rate of {samplerate} Hz.")

WAV file imported successfully!!! This file has been created with a sample rate of 44100 Hz.


In [8]:
# just for testing | at the end we can delete it
print(f"We will work with {type(data)}")

We will work with <class 'numpy.ndarray'>


## **Coder functions**

### make_mp3_analysisfb

In [15]:
def make_mp3_analysisfb(h, M):
    L = 10
    H = np.ndarray(L, M)
    # with a for loop call frame_sub_analysis() to compute the coefficients
    for i in range(0, M):
        xbuff = data[i] # wrooooooooong
        q = 5 # wrooooooooong
        H[i] = frame_sub_analysis(xbuff, H, q) # something like it
    return H

## **Decoder functions**

### make_mp3_synthesisfb

In [13]:
def make_mp3_synthesisfb(h, M):
    G = 0
    # with a for loop call frame_sub_synthesis() to compute the coefficients
    return G

## **Coder - Decoder**

### 1) Subband filtering

1.1: `H and G computation`

In [14]:
h = h_coeffs
M = 32
H = make_mp3_analysisfb(h, M)

In [16]:
h = h_coeffs
M = 32
G = make_mp3_synthesisfb(h, M)

1.2: `Hs plotting on f frequency(Hz)`

1.3: `Hs plotting on z frequency(barks)`

1.4: `codec0`

1.5: `coder0`

1.6: `decoder0`

1.7: `Coding/Decoding`